# Import Libraries

In [1]:
import sys
sys.path.append("..")
import seaborn as sb
import dgl
import torch

import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from tqdm import tqdm
import torch.nn.functional as F

import statistics
import argparse
import random

from dgl.data import CitationGraphDataset
# from dgl.nn import GINConv, GraphConv, SAGEConv
import seaborn as sb
import torch
import torch.nn as nn
from torch.autograd import Variable
from scipy.optimize import linear_sum_assignment
import scipy
import scipy.optimize
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import sklearn as sk
import networkx as nx


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import random
import math


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data

from torch_geometric.nn import GCNConv,GINConv,SAGEConv,GATConv,PNAConv, GraphSAGE
from torch_geometric.utils import add_self_loops
from torch_geometric.transforms import normalize_features
from pygod.utils import load_data
from pygod.utils.utility import check_parameter
from pygod.metrics import eval_roc_auc
from pygod.generator import gen_contextual_outliers, gen_structural_outliers
from scipy.linalg import sqrtm

/home/koo/miniconda3/envs/gadnr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/koo/miniconda3/envs/gadnr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/koo/miniconda3/envs/gadnr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  F

# Utils

In [2]:
def safe_normalize(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    if max_val == min_val:
        return torch.zeros_like(tensor)  # or tensor.clone().fill_(0)
    return (tensor - min_val) / (max_val - min_val)
def gen_joint_structural_outliers(data, m, n, random_state=None):
    """
    We randomly select n nodes from the network which will be the anomalies 
    and for each node we select m nodes from the network. 
    We connect each of n nodes with the m other nodes.

    Parameters
    ----------
    data : PyTorch Geometric Data instance (torch_geometric.data.Data)
        The input data.
    m : int
        Number nodes in the outlier cliques.
    n : int
        Number of outlier cliques.
    p : int, optional
        Probability of edge drop in cliques. Default: ``0``.
    random_state : int, optional
        The seed to control the randomness, Default: ``None``.

    Returns
    -------
    data : PyTorch Geometric Data instance (torch_geometric.data.Data)
        The structural outlier graph with injected edges.
    y_outlier : torch.Tensor
        The outlier label tensor where 1 represents outliers and 0 represents
        regular nodes.
    """

    if not isinstance(data, Data):
        raise TypeError("data should be torch_geometric.data.Data")

    if isinstance(m, int):
        check_parameter(m, low=0, high=data.num_nodes, param_name='m')
    else:
        raise ValueError("m should be int, got %s" % m)

    if isinstance(n, int):
        check_parameter(n, low=0, high=data.num_nodes, param_name='n')
    else:
        raise ValueError("n should be int, got %s" % n)

    check_parameter(m * n, low=0, high=data.num_nodes, param_name='m*n')

    if random_state:
        np.random.seed(random_state)


    outlier_idx = np.random.choice(data.num_nodes, size=n, replace=False)
    all_nodes = [i for i in range(data.num_nodes)]
    rem_nodes = []
    
    for node in all_nodes:
        if node is not outlier_idx:
            rem_nodes.append(node)
    
    
    
    new_edges = []
    
    # connect all m nodes in each clique
    for i in range(0, n):
        other_idx = np.random.choice(data.num_nodes, size=m, replace=False)
        for j in other_idx:
            new_edges.append(torch.tensor([[i, j]], dtype=torch.long))
                    

    new_edges = torch.cat(new_edges)


    y_outlier = torch.zeros(data.x.shape[0], dtype=torch.long)
    outlier_idx = torch.tensor(outlier_idx, dtype=torch.long)  # 👈 numpy → torch 변환
    y_outlier[outlier_idx] = 1
    
    data.edge_index = torch.cat([data.edge_index, new_edges.T], dim=1)

    return data, y_outlier


def KL_neighbor_loss(predictions, targets, mask_len):
    x1 = predictions.squeeze().cpu().detach()
    x2 = targets.squeeze().cpu().detach()
    
    mean_x1 = x1.mean(0)
    mean_x2 = x2.mean(0)
    
    nn = x1.shape[0]
    h_dim = x1.shape[1]
    
    cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / max((nn-1),1)
    cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / max((nn-1),1)
    
    eye = torch.eye(h_dim)
    cov_x1 = cov_x1 + eye
    cov_x2 = cov_x2 + eye
    
    KL_loss = 0.5 * (math.log(torch.det(cov_x1) / torch.det(cov_x2)) - h_dim  + torch.trace(torch.inverse(cov_x2).matmul(cov_x1)) 
            + (mean_x2 - mean_x1).reshape(1,-1).matmul(torch.inverse(cov_x2)).matmul(mean_x2 - mean_x1))
    KL_loss = KL_loss.to(device)
    return KL_loss

def W2_neighbor_loss(predictions, targets, mask_len):
    
    x1 = predictions.squeeze().cpu().detach()
    x2 = targets.squeeze().cpu().detach()
    
    mean_x1 = x1.mean(0)
    mean_x2 = x2.mean(0)

    nn = x1.shape[0]
    
    cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / (nn-1)
    cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / (nn-1)
    

    W2_loss = torch.square(mean_x1-mean_x2).sum() + torch.trace(cov_x1 + cov_x2 
                     + 2 * sqrtm(sqrtm(cov_x1) @ (cov_x2.numpy()) @ (sqrtm(cov_x1))))

    return W2_loss



# Layers

In [3]:
class MLP(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        

        super(MLP, self).__init__()

        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            for layer in range(num_layers - 1):
                self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for layer in range(self.num_layers - 1):
                h = self.linears[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 0, 1)
                    h = torch.transpose(h, 1, 2)
                    
                h = self.batch_norms[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 1, 2)
                    h = torch.transpose(h, 0, 1)

                h = F.relu(h)
                # h = F.relu(self.linears[layer](h))
                
            return self.linears[self.num_layers - 1](h)


class MLP_generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_generator, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.linear2 = nn.Linear(output_dim, output_dim)
        self.linear3 = nn.Linear(output_dim, output_dim)
        self.linear4 = nn.Linear(output_dim, output_dim)

    def forward(self, embedding):
        neighbor_embedding = F.relu(self.linear(embedding))
        neighbor_embedding = F.relu(self.linear2(neighbor_embedding))
        neighbor_embedding = F.relu(self.linear3(neighbor_embedding))
        neighbor_embedding = self.linear4(neighbor_embedding)
        return neighbor_embedding


class PairNorm(nn.Module):
    def __init__(self, mode='PN', scale=10):
        
        assert mode in ['None', 'PN', 'PN-SI', 'PN-SCS']
        super(PairNorm, self).__init__()
        self.mode = mode
        self.scale = scale

        # Scale can be set based on origina data, and also the current feature lengths.
        # We leave the experiments to future. A good pool we used for choosing scale:
        # [0.1, 1, 10, 50, 100]
    def forward(self, x):
        if self.mode == 'None':
            return x
        col_mean = x.mean(dim=0)
        if self.mode == 'PN':
            x = x - col_mean
            rownorm_mean = (1e-6 + x.pow(2).sum(dim=1).mean()).sqrt()
            x = self.scale * x / rownorm_mean
        if self.mode == 'PN-SI':
            x = x - col_mean
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual
        if self.mode == 'PN-SCS':
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual - col_mean
        return x


# FNN
class FNN(nn.Module):
    def __init__(self, in_features, hidden, out_features, layer_num):
        super(FNN, self).__init__()
        self.linear1 = MLP(layer_num, in_features, hidden, out_features)
        self.linear2 = nn.Linear(out_features, out_features)
    def forward(self, embedding):
        x = self.linear1(embedding)
        x = self.linear2(F.relu(x))
        x = F.relu(x)
        return x


# GAD-NR

In [6]:

# Training
def train(data, y, yc, ys, yj, ysj, lr, epoch, device, encoder, lambda_loss1, lambda_loss2, lambda_loss3, hidden_dim, sample_size=10,loss_step=20,
          real_loss=False,calculate_contextual=False,calculate_structural=False):
    '''
     Main training function
     INPUT:
     -----------------------
     data : torch geometric dataset object
     lr    :    learning rate
     epoch     :    number of training epoch
     device     :   CPU or GPU
     encoder    :    GCN or GIN or GraphSAGE
     lambda_loss    :   Trade-off between degree loss and neighborhood reconstruction loss
     hidden_dim     :   latent variable dimension
    '''
    
    in_nodes = data.edge_index[0,:]
    out_nodes = data.edge_index[1,:]
    
    
    neighbor_dict = {}
    for in_node, out_node in zip(in_nodes, out_nodes):
        if in_node.item() not in neighbor_dict:
            neighbor_dict[in_node.item()] = []
        neighbor_dict[in_node.item()].append(out_node.item())

    neighbor_num_list = []
    for i in neighbor_dict:
        neighbor_num_list.append(len(neighbor_dict[i]))
    
    neighbor_num_list = torch.tensor(neighbor_num_list).to(device)
    
    in_dim = data.x.shape[1]
    GNNModel = GNNStructEncoder(in_dim, hidden_dim, hidden_dim, 2, sample_size, device=device, 
                    neighbor_num_list=neighbor_num_list, GNN_name=encoder, 
                    lambda_loss1=lambda_loss1, lambda_loss2=lambda_loss2,lambda_loss3=lambda_loss3)
    GNNModel.to(device)
    degree_params = list(map(id, GNNModel.degree_decoder.parameters()))
    base_params = filter(lambda p: id(p) not in degree_params,
                         GNNModel.parameters())

    opt = torch.optim.Adam([{'params': base_params}, {'params': GNNModel.degree_decoder.parameters(), 'lr': 1e-2}],lr=lr, weight_decay=0.0003)
    min_loss = float('inf')
    arg_min_loss_per_node = None
    
    best_auc = 0
    best_auc_contextual = 0
    best_auc_dense_structural = 0
    best_auc_joint_structural = 0
    best_auc_structure_type = 0
    
        
    loss_values = []
    for i in tqdm(range(1,epoch+1,1)):
        
        if i%loss_step==0:
            GNNModel.lambda_loss2 = GNNModel.lambda_loss2 + 0.5
            GNNModel.lambda_loss3 = GNNModel.lambda_loss3 / 2
        
        loss,loss_per_node,h_loss,degree_loss,feature_loss = GNNModel(data.edge_index, data.x, neighbor_num_list, neighbor_dict, device=device)
        
        
        
        loss_per_node = loss_per_node.cpu().detach()
        
        h_loss = h_loss.cpu().detach()
        degree_loss = degree_loss.cpu().detach()
        feature_loss = feature_loss.cpu().detach()
        
        h_loss_norm = safe_normalize(h_loss)
        degree_loss_norm = safe_normalize(degree_loss)
        feature_loss_norm = safe_normalize(feature_loss)
        
        comb_loss = args.h_loss_weight * h_loss_norm + args.degree_loss_weight *  degree_loss_norm + args.feature_loss_weight * feature_loss_norm
        
        if real_loss:
            comp_loss = loss_per_node
        else:
            comp_loss = comb_loss
            
        
        auc_score = eval_roc_auc(y.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score(benchmark/combined): ", auc_score)
        
        contextual_auc_score = eval_roc_auc(yc.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (contextual): ", contextual_auc_score)

        dense_structural_auc_score = eval_roc_auc(ys.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (structural): ", dense_structural_auc_score)
        
        joint_type_auc_score = eval_roc_auc(yj.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (joint-type): ", joint_type_auc_score)
        
        structure_type_auc_score = eval_roc_auc(ysj.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (structure type): ", structure_type_auc_score) 
        
        best_auc = max(best_auc, auc_score)
        best_auc_contextual = max(best_auc_contextual, contextual_auc_score)
        best_auc_dense_structural = max(best_auc_dense_structural, dense_structural_auc_score)
        best_auc_joint_type = max(best_auc_joint_structural, joint_type_auc_score)
        best_auc_structure_type = max(best_auc_structure_type, structure_type_auc_score)
        
        
        
        print("===========================================================================================")
        print("Dataset Name: ",dataset_str, " Best AUC Score(benchmark/combined): ", best_auc)
        
        # print("Dataset Name: ",dataset_str, " Best AUC Score (contextual): ", best_auc_contextual)

        
        # print("Dataset Name: ",dataset_str, " Best AUC Score (structural): ", best_auc_dense_structural)
        
        
        # print("Dataset Name: ",dataset_str, " Best AUC Score (joint-type): ", best_auc_joint_type)
        
        
        # print("Dataset Name: ",dataset_str, " Best AUC Score (structure type): ", best_auc_structure_type) 
        print("===========================================================================================")
        
        
        if loss < min_loss:
            min_loss = loss
            arg_min_loss_per_node = loss_per_node
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        loss = loss.cpu().detach()
        loss_values.append(loss)
        
        
        if args.plot_loss:

            plt.plot(np.array(loss_values), 'r')
            plt.show()
    
        return_value = {
        "loss": min_loss.item(),
        "loss_per_node": arg_min_loss_per_node.cpu().detach(),
        "best_auc": best_auc,
        "best_auc_contextual": best_auc_contextual,
        "best_auc_dense_structural": best_auc_dense_structural,
        "best_auc_joint_type": best_auc_joint_type,
        "best_auc_structure_type": best_auc_structure_type,
        "loss_values": loss_values
        }
            
    
    return min_loss.item(), arg_min_loss_per_node.cpu().detach(), return_value




def evaluate(model, embeddings, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(embeddings)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train_real_datasets(dataset_str, epoch_num = 10, lr = 5e-6, encoder = "GCN", 
                        lambda_loss1=1e-2, lambda_loss2=1e-3, lambda_loss3=1e-3, sample_size=8, loss_step=20, hidden_dim=None,
                        real_loss=False,calculate_contextual=False,calculate_structural=False):
    
    data = load_data(dataset_str)
    node_features = data.x
    
    if args.normalize_feat:
    
        node_features_min = node_features.min()
        node_features_max = node_features.max()
        node_features = (node_features - node_features_min)/node_features_max
        data.x = node_features
    
    yc = []
    ys = []
    yj = []
    
    if calculate_contextual:
        
        if dataset_str == "inj_cora":
            yc = data.y >> 0 & 1 # contextual outliers
        else:
            data, yc = gen_contextual_outliers(data=data,n=args.contextual_n,k=args.contextual_k)
            
        yc = yc.cpu().detach()
    
    
    if calculate_structural:
        
        if dataset_str == "inj_cora":
            ys = data.y >> 1 & 1 # structural outliers
        else:
            data, ys = gen_structural_outliers(data=data,n=args.structural_n,m=args.structural_m,p=0.2)
            
        ys = ys.cpu().detach()
        data, yj = gen_joint_structural_outliers(data=data,n=args.structural_n,m=args.structural_m)
        
    
    if args.use_combine_outlier:
        data.y = torch.logical_or(ys, yc).int()
        
    ysj = torch.logical_or(ys, yj).int()
    y = data.y.bool()    # binary labels (inlier/outlier)
    y = y.cpu().detach()
    
    edge_index = data.edge_index.cpu()
    
    num_nodes = node_features.shape[0]
    self_edges = torch.tensor([[i for i in range(num_nodes)],[i for i in range(num_nodes)]])
    edge_index = torch.cat([edge_index,self_edges],dim=1)
    data.edge_index = edge_index
    data = data.to(device)
    

    loss, loss_per_node, result = train(data, y, yc, ys, yj, ysj, lr=lr, epoch=epoch_num, device=device, encoder=encoder, lambda_loss1=lambda_loss1, 
          lambda_loss2=lambda_loss2, lambda_loss3=lambda_loss3, hidden_dim=hidden_dim, sample_size=sample_size,loss_step=loss_step,
                                 real_loss=real_loss,calculate_contextual=calculate_contextual,calculate_structural=calculate_structural)
    
    from datetime import datetime
    import os

    # 시간 기반 디렉토리 생성
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"results/{dataset_str}_{timestamp}"
    os.makedirs(save_dir, exist_ok=True)


    # 저장
    with open(f"{save_dir}/result.txt", "w") as f:
        f.write(f"Dataset: {dataset_str}\n")
        f.write(f"Best AUC (benchmark): {result['best_auc']:.2f}\n")
        f.write(f"Contextual AUC: {result['best_auc_contextual']:.2f}\n")
        f.write(f"Structural AUC: {result['best_auc_dense_structural']:.2f}\n")
        f.write(f"Joint-Type AUC: {result['best_auc_joint_type']:.2f}\n")
        f.write(f"Structure-Type AUC: {result['best_auc_structure_type']:.2f}\n")
        f.write(f"Final Loss: {result['loss']:.4f}\n")

In [7]:
# generate ground truth neighbors Hv
def generate_gt_neighbor(neighbor_dict, node_embeddings, neighbor_num_list, in_dim):
    max_neighbor_num = max(neighbor_num_list)
    all_gt_neighbor_embeddings = []
    for i, embedding in enumerate(node_embeddings):
        neighbor_indexes = neighbor_dict[i]
        neighbor_embeddings = []
        for index in neighbor_indexes:
            neighbor_embeddings.append(node_embeddings[index].tolist())
        if len(neighbor_embeddings) < max_neighbor_num:
            for _ in range(max_neighbor_num - len(neighbor_embeddings)):
                neighbor_embeddings.append(torch.zeros(in_dim).tolist())
        all_gt_neighbor_embeddings.append(neighbor_embeddings)
    return all_gt_neighbor_embeddings


# Main Autoencoder structure here
class GNNStructEncoder(nn.Module):
    def __init__(self, in_dim0, in_dim, hidden_dim, layer_num, sample_size, device, neighbor_num_list, 
                 GNN_name="GIN", norm_mode="PN-SCS", norm_scale=20, lambda_loss1=0.01, lambda_loss2=0.001, lambda_loss3=0.0001):
        
        super(GNNStructEncoder, self).__init__()
        
        self.mlp0 = nn.Linear(in_dim0, hidden_dim)
        self.norm = PairNorm(norm_mode, norm_scale)
        self.out_dim = hidden_dim
        self.lambda_loss1 = lambda_loss1
        self.lambda_loss2 = lambda_loss2
        self.lambda_loss3 = lambda_loss3
        # GNN Encoder
        if GNN_name == "GIN":
            self.linear1 = MLP(layer_num, hidden_dim, hidden_dim, hidden_dim)
            self.graphconv1 = GINConv(self.linear1)
            self.linear2 = MLP(layer_num, hidden_dim, hidden_dim, hidden_dim)
            self.graphconv2 = GINConv(self.linear2)
        elif GNN_name == "GCN":
            self.graphconv1 = GCNConv(hidden_dim, hidden_dim)
            self.graphconv2 = GCNConv(hidden_dim, hidden_dim)
        elif GNN_name == "GAT":
            self.graphconv1 = GATConv(hidden_dim, hidden_dim)
            self.graphconv2 = GATConv(hidden_dim, hidden_dim)
        else:
            self.graphconv1 = SAGEConv(hidden_dim, hidden_dim, aggr='mean')
            # self.graphconv2 = SAGEConv(hidden_dim, hidden_dim, aggr='mean')
            
            # self.graphconv1 = GraphSAGE(hidden_dim, hidden_dim, aggr='mean', num_layers=1)
            

        self.neighbor_num_list = neighbor_num_list
        self.tot_node = len(neighbor_num_list)

        self.gaussian_mean = nn.Parameter(
            torch.FloatTensor(sample_size, hidden_dim).uniform_(-0.5 / hidden_dim,
                                                                                     0.5 / hidden_dim)).to(device)
        self.gaussian_log_sigma = nn.Parameter(
            torch.FloatTensor(sample_size, hidden_dim).uniform_(-0.5 / hidden_dim,
                                                                                     0.5 / hidden_dim)).to(device)
        self.m = torch.distributions.Normal(torch.zeros(sample_size, hidden_dim),
                                            torch.ones(sample_size, hidden_dim))
        
        self.m_batched = torch.distributions.Normal(torch.zeros(sample_size, self.tot_node, hidden_dim),
                                            torch.ones(sample_size, self.tot_node, hidden_dim))

        self.m_h = torch.distributions.Normal(torch.zeros(sample_size, hidden_dim),
                                            50* torch.ones(sample_size, hidden_dim))

        # Before MLP Gaussian Means, and std

        self.mlp_gaussian_mean = nn.Parameter(
            torch.FloatTensor(hidden_dim).uniform_(-0.5 / hidden_dim, 0.5 / hidden_dim)).to(device)
        self.mlp_gaussian_log_sigma = nn.Parameter(
            torch.FloatTensor(hidden_dim).uniform_(-0.5 / hidden_dim, 0.5 / hidden_dim)).to(device)
        self.mlp_m = torch.distributions.Normal(torch.zeros(hidden_dim), torch.ones(hidden_dim))

        self.mlp_mean = FNN(hidden_dim, hidden_dim, hidden_dim, 3)
        self.mlp_sigma = FNN(hidden_dim, hidden_dim, hidden_dim, 3)
        self.softplus = nn.Softplus()

        # self.mean_agg = SAGEConv(hidden_dim, hidden_dim, aggr='mean', normalize = True)
        self.mean_agg = GraphSAGE(hidden_dim, hidden_dim, aggr='mean', num_layers=1)
        self.std_agg = PNAConv(hidden_dim, hidden_dim, aggregators=["std"],scalers=["identity"], deg=neighbor_num_list)        
        self.layer1_generator = MLP_generator(hidden_dim, hidden_dim)
        
        # Decoders
        self.degree_decoder = FNN(hidden_dim, hidden_dim, 1, 4)
        self.feature_decoder = FNN(hidden_dim, hidden_dim, in_dim, 3)
        self.degree_loss_func = nn.MSELoss()
        self.feature_loss_func = nn.MSELoss()
        self.pool = mp.Pool(4)
        self.in_dim = in_dim
        self.sample_size = sample_size 
        self.init_projection = FNN(in_dim, hidden_dim, hidden_dim, 1)
        

    def forward_encoder(self, x, edge_index):
        
        # Apply graph convolution and activation, pair-norm to avoid trivial solution
        h0 = self.mlp0(x)
        l1 = self.graphconv1(h0, edge_index)
        return l1, h0
        
        

    # Sample neighbors from neighbor set, if the length of neighbor set less than sample size, then do the padding.
    def sample_neighbors(self, indexes, neighbor_dict, gt_embeddings):
        sampled_embeddings_list = []
        mark_len_list = []
        for index in indexes:
            sampled_embeddings = []
            neighbor_indexes = neighbor_dict[index]
            if len(neighbor_indexes) < self.sample_size:
                mask_len = len(neighbor_indexes)
                sample_indexes = neighbor_indexes
            else:
                sample_indexes = random.sample(neighbor_indexes, self.sample_size)
                mask_len = self.sample_size
            for index in sample_indexes:
                sampled_embeddings.append(gt_embeddings[index].tolist())
            if len(sampled_embeddings) < self.sample_size:
                for _ in range(self.sample_size - len(sampled_embeddings)):
                    sampled_embeddings.append(torch.zeros(self.out_dim).tolist())
            sampled_embeddings_list.append(sampled_embeddings)
            mark_len_list.append(mask_len)
        
        return sampled_embeddings_list, mark_len_list
    
    def reconstruction_neighbors2(self, l1, h0, edge_index):
                
        recon_loss = 0
        recon_loss_per_node = []
    
        sample_sz_per_node = [self.sample_size]* self.tot_node

        # mean_neigh = self.graphconv1(h0, edge_index)
        # mean_neigh = self.graphconv2(mean_neigh, edge_index)
        # mean_neigh = l1
        mean_neigh = self.mean_agg(h0, edge_index).detach()
        std_neigh = self.std_agg(h0, edge_index).detach()
        
        cov_neigh = torch.bmm(std_neigh.unsqueeze(dim=-1),std_neigh.unsqueeze(dim=1))
        
        target_mean = mean_neigh
        target_cov = cov_neigh
        
        self_embedding = l1
        # self_embedding = _normalize(self_embedding)
        
        self_embedding = self_embedding.unsqueeze(0)
        self_embedding = self_embedding.repeat(self.sample_size, 1, 1)
        generated_mean = self.mlp_mean(self_embedding)
        generated_sigma = self.mlp_sigma(self_embedding)

        
        std_z = self.m_batched.sample().to(device)
        var = generated_mean + generated_sigma.exp() * std_z
        nhij = self.layer1_generator(var)
        
        generated_mean = torch.mean(nhij,dim=0)
        generated_std = torch.std(nhij,dim=0)
        generated_cov = torch.bmm(generated_std.unsqueeze(dim=-1),generated_std.unsqueeze(dim=1))/self.sample_size
        
        
        tot_nodes = l1.shape[0]
        h_dim = l1.shape[1]
        
        single_eye = torch.eye(h_dim).to(device)
        single_eye = single_eye.unsqueeze(dim=0)
        batch_eye = single_eye.repeat(tot_nodes,1,1)
        
        target_cov = target_cov + batch_eye
        generated_cov = generated_cov + batch_eye

        
        det_target_cov = torch.linalg.det(target_cov) 
        det_generated_cov = torch.linalg.det(generated_cov) 
        trace_mat = torch.matmul(torch.inverse(generated_cov),target_cov)
             
        
        x = torch.bmm(torch.unsqueeze(generated_mean - target_mean,dim=1),torch.inverse(generated_cov))
        y = torch.unsqueeze(generated_mean - target_mean,dim=-1)
        z = torch.bmm(x,y).squeeze()
        
        KL_loss = 0.5 * (torch.log(det_target_cov / det_generated_cov) - h_dim  + trace_mat.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) + z)
        
        recon_loss = torch.mean(KL_loss)
        recon_loss_per_node = KL_loss
        
        
        return recon_loss, recon_loss_per_node
    
                
        # self_embedding_min = self_embedding.min()
        # self_embedding_max = self_embedding.max()
        # self_embedding = (self_embedding - self_embedding_min)/(self_embedding_max+1e-14)
        # generated_mean = self.mean_generator(self_embedding)
        # generated_mean = _normalize(generated_mean)
        # generated_std = self.std_generator(self_embedding)
        # generated_std = _normalize(generated_std)
        # generated_std = generated_std.abs().squeeze()
        # generated_std = self.softplus(generated_std)
        # generated_std = generated_std.exp().squeeze()
        
        # cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / max((nn-1),1)
        # cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / max((nn-1),1)
        
        # generated_cov = generated_cov + torch.rand(generated_cov.shape).to(device)
        # generated_cov = generated_cov * batch_eye
        
        # print(torch.isfinite(det_target_cov).all())
        # print(torch.isfinite(det_generated_cov).all()) 
        
        # KL_loss = 0.5 * (math.log(torch.det(cov_x1) / torch.det(cov_x2)) - h_dim  + torch.trace(torch.inverse(cov_x2).matmul(cov_x1)) 
        #     + (mean_x2 - mean_x1).reshape(1,-1).matmul(torch.inverse(cov_x2)).matmul(mean_x2 - mean_x1))
    

    def reconstruction_neighbors(self, FNN_generator, neighbor_indexes, neighbor_dict, from_layer, to_layer, device):
        
        
        local_index_loss = 0
        local_index_loss_per_node = []
        sampled_embeddings_list, mark_len_list = self.sample_neighbors(neighbor_indexes, neighbor_dict, to_layer)
        for i, neighbor_embeddings1 in enumerate(sampled_embeddings_list):
            # Generating h^k_v, reparameterization trick
            
            # print(len(neighbor_embeddings1))
            
            index = neighbor_indexes[i]
            mask_len1 = mark_len_list[i]
            mean = from_layer[index].repeat(self.sample_size, 1)
            mean = self.mlp_mean(mean)
            sigma = from_layer[index].repeat(self.sample_size, 1)
            sigma = self.mlp_sigma(sigma)
            std_z = self.m.sample().to(device)
            var = mean + sigma.exp() * std_z
            nhij = FNN_generator(var)
            
            generated_neighbors = nhij
            sum_neighbor_norm = 0
            
            for indexi, generated_neighbor in enumerate(generated_neighbors):
                sum_neighbor_norm += torch.norm(generated_neighbor) / math.sqrt(self.out_dim)
            generated_neighbors = torch.unsqueeze(generated_neighbors, dim=0).to(device)
            target_neighbors = torch.unsqueeze(torch.FloatTensor(neighbor_embeddings1), dim=0).to(device)
            
            if args.neigh_loss == "KL":
                KL_loss = KL_neighbor_loss(generated_neighbors, target_neighbors, mask_len1)
                local_index_loss += KL_loss
                local_index_loss_per_node.append(KL_loss)
            
            else:
                W2_loss = W2_neighbor_loss(generated_neighbors, target_neighbors, mask_len1)
                local_index_loss += W2_loss
                local_index_loss_per_node.append(W2_loss)
            
            
        local_index_loss_per_node = torch.stack(local_index_loss_per_node)
        return local_index_loss, local_index_loss_per_node
    

    def neighbor_decoder(self, gij, ground_truth_degree_matrix, h0, neighbor_dict, device, h, edge_index):
        
        # Degree decoder below:
        tot_nodes = gij.shape[0]
        degree_logits = self.degree_decoding(gij)
        ground_truth_degree_matrix = torch.unsqueeze(ground_truth_degree_matrix, dim=1)
        degree_loss = self.degree_loss_func(degree_logits, ground_truth_degree_matrix.float())
        degree_loss_per_node = (degree_logits-ground_truth_degree_matrix).pow(2)
        _, degree_masks = torch.max(degree_logits.data, dim=1)
        h_loss = 0
        feature_loss = 0
        # layer 1
        loss_list = []
        loss_list_per_node = []
        feature_loss_list = []
        # Sample multiple times to remove noise
        for _ in range(3):
            local_index_loss_sum = 0
            local_index_loss_sum_per_node = []
            indexes = []
            h0_prime = self.feature_decoder(gij)
            feature_losses = self.feature_loss_func(h0, h0_prime)
            feature_losses_per_node = (h0-h0_prime).pow(2).mean(1)
            feature_loss_list.append(feature_losses_per_node)
            

            
            # local_index_loss, local_index_loss_per_node = self.reconstruction_neighbors(self.layer1_generator, indexes, neighbor_dict, gij, h0, device)
            local_index_loss, local_index_loss_per_node = self.reconstruction_neighbors2(gij,h0,edge_index)
            
            # print(local_index_loss, local_index_loss2)
            # print(local_index_loss_per_node, local_index_loss_per_node2)

            loss_list.append(local_index_loss)
            loss_list_per_node.append(local_index_loss_per_node)
            
        loss_list = torch.stack(loss_list)
        h_loss += torch.mean(loss_list)
        
        loss_list_per_node = torch.stack(loss_list_per_node)
        h_loss_per_node = torch.mean(loss_list_per_node,dim=0)
        
        feature_loss_per_node = torch.mean(torch.stack(feature_loss_list),dim=0)
        feature_loss += torch.mean(torch.stack(feature_loss_list))
                
        h_loss_per_node = h_loss_per_node.reshape(tot_nodes,1)
        degree_loss_per_node = degree_loss_per_node.reshape(tot_nodes,1)
        feature_loss_per_node = feature_loss_per_node.reshape(tot_nodes,1)
        
        loss = self.lambda_loss1 * h_loss + degree_loss * self.lambda_loss3 + self.lambda_loss2 * feature_loss
        loss_per_node = self.lambda_loss1 * h_loss_per_node + degree_loss_per_node * self.lambda_loss3 + self.lambda_loss2 * feature_loss_per_node
        

        return loss,loss_per_node,h_loss_per_node,degree_loss_per_node,feature_loss_per_node

    def degree_decoding(self, node_embeddings):
        degree_logits = F.relu(self.degree_decoder(node_embeddings))
        return degree_logits

    def forward(self, edge_index, x, ground_truth_degree_matrix, neighbor_dict, device):
        
        # Generate GNN encodings
        l1, h0 = self.forward_encoder(x, edge_index)
        loss, loss_per_node,h_loss,degree_loss,feature_loss = self.neighbor_decoder(l1, ground_truth_degree_matrix, h0, neighbor_dict, device, x, edge_index)
        
        return loss, loss_per_node,h_loss,degree_loss,feature_loss


# Execution

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 1/500 [00:01<11:56,  1.44s/it]

Dataset Name:  enron , AUC Score(benchmark/combined):  71.39562389118865
Dataset Name:  enron , AUC Score (contextual):  41.47326927814733
Dataset Name:  enron , AUC Score (structural):  48.74533115371279
Dataset Name:  enron , AUC Score (joint-type):  48.524267060852424
Dataset Name:  enron , AUC Score (structure type):  48.73624331818312
Dataset Name:  enron  Best AUC Score(benchmark/combined):  71.39562389118865
Dataset Name:  enron , AUC Score(benchmark/combined):  71.71643997634536
Dataset Name:  enron , AUC Score (contextual):  42.5030795762503
Dataset Name:  enron , AUC Score (structural):  51.54252736909892
Dataset Name:  enron , AUC Score (joint-type):  41.39443212613944


  1%|          | 3/500 [00:01<03:42,  2.23it/s]

Dataset Name:  enron , AUC Score (structure type):  51.15070176943086
Dataset Name:  enron  Best AUC Score(benchmark/combined):  71.71643997634536
Dataset Name:  enron , AUC Score(benchmark/combined):  69.60969840331165
Dataset Name:  enron , AUC Score (contextual):  18.073417097807337
Dataset Name:  enron , AUC Score (structural):  50.01832862733442
Dataset Name:  enron , AUC Score (joint-type):  54.56270017245627
Dataset Name:  enron , AUC Score (structure type):  50.19409427436183
Dataset Name:  enron  Best AUC Score(benchmark/combined):  71.71643997634536
Dataset Name:  enron , AUC Score(benchmark/combined):  67.93169722057955
Dataset Name:  enron , AUC Score (contextual):  18.913525498891353


...

Dataset Name:  enron , AUC Score (structure type):  31.110682331418115
Dataset Name:  enron  Best AUC Score(benchmark/combined):  72.42460082791249
Dataset Name:  enron , AUC Score(benchmark/combined):  36.596688350088705
Dataset Name:  enron , AUC Score (contextual):  83.31608770633161
Dataset Name:  enron , AUC Score (structural):  30.373309555654625
Dataset Name:  enron , AUC Score (joint-type):  47.97610248829761
Dataset Name:  enron , AUC Score (structure type):  31.045698386835518
Dataset Name:  enron  Best AUC Score(benchmark/combined):  72.42460082791249
Dataset Name:  enron , AUC Score(benchmark/combined):  37.563571850975755
Dataset Name:  enron , AUC Score (contextual):  83.35550628233554
Dataset Name:  enron , AUC Score (structural):  30.316986179224255
Dataset Name:  enron , AUC Score (joint-type):  48.3690564178369
Dataset Name:  enron , AUC Score (structure type):  31.006727076961194
Dataset Name:  enron  Best AUC Score(benchmark/combined):  72.42460082791249


100%|█████████▉| 499/500 [01:45<00:00,  4.82it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  37.650059136605556
Dataset Name:  enron , AUC Score (contextual):  83.34072431633408
Dataset Name:  enron , AUC Score (structural):  30.361916084608907
Dataset Name:  enron , AUC Score (joint-type):  48.359201773835906
Dataset Name:  enron , AUC Score (structure type):  31.04955740407245
Dataset Name:  enron  Best AUC Score(benchmark/combined):  72.42460082791249
Dataset Name:  enron , AUC Score(benchmark/combined):  37.548787699586036
Dataset Name:  enron , AUC Score (contextual):  83.30623306233062
Dataset Name:  enron , AUC Score (structural):  30.43275375241492


100%|██████████| 500/500 [01:46<00:00,  4.72it/s]

Dataset Name:  enron , AUC Score (joint-type):  48.377679231337765
Dataset Name:  enron , AUC Score (structure type):  31.118400365892
Dataset Name:  enron  Best AUC Score(benchmark/combined):  72.42460082791249


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 1/500 [00:00<01:47,  4.62it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  56.72678888231816
Dataset Name:  enron , AUC Score (contextual):  37.926829268292686
Dataset Name:  enron , AUC Score (structural):  63.22980135730916
Dataset Name:  enron , AUC Score (joint-type):  88.06848977580685
Dataset Name:  enron , AUC Score (structure type):  64.19613336001296
Dataset Name:  enron  Best AUC Score(benchmark/combined):  56.72678888231816
Dataset Name:  enron , AUC Score(benchmark/combined):  62.888823181549384
Dataset Name:  enron , AUC Score (contextual):  36.646957378664695
Dataset Name:  enron , AUC Score (structural):  63.25902808738297
Dataset Name:  enron , AUC Score (joint-type):  78.01675289480168


  1%|          | 3/500 [00:00<01:59,  4.15it/s]

Dataset Name:  enron , AUC Score (structure type):  63.835481996017116
Dataset Name:  enron  Best AUC Score(benchmark/combined):  62.888823181549384
Dataset Name:  enron , AUC Score(benchmark/combined):  51.114725014784156
Dataset Name:  enron , AUC Score (contextual):  30.404040404040405
Dataset Name:  enron , AUC Score (structural):  62.79501659483826
Dataset Name:  enron , AUC Score (joint-type):  71.47080561714708
Dataset Name:  enron , AUC Score (structure type):  63.136047032368104
Dataset Name:  enron  Best AUC Score(benchmark/combined):  62.888823181549384


...

Dataset Name:  enron , AUC Score (structural):  54.86773666220836
Dataset Name:  enron , AUC Score (joint-type):  61.88470066518846
Dataset Name:  enron , AUC Score (structure type):  55.14121144555927
Dataset Name:  enron  Best AUC Score(benchmark/combined):  78.38187463039621
Dataset Name:  enron , AUC Score(benchmark/combined):  63.32569485511532
Dataset Name:  enron , AUC Score (contextual):  18.492239467849227
Dataset Name:  enron , AUC Score (structural):  53.299053846534896
Dataset Name:  enron , AUC Score (joint-type):  57.304754865730466
Dataset Name:  enron , AUC Score (structure type):  53.455392619272224
Dataset Name:  enron  Best AUC Score(benchmark/combined):  78.38187463039621


100%|█████████▉| 499/500 [01:43<00:00,  4.99it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  73.92001774098166
Dataset Name:  enron , AUC Score (contextual):  23.88642522788864
Dataset Name:  enron , AUC Score (structural):  49.16312478327637
Dataset Name:  enron , AUC Score (joint-type):  79.55900468095591
Dataset Name:  enron , AUC Score (structure type):  50.33835481996017
Dataset Name:  enron  Best AUC Score(benchmark/combined):  78.38187463039621


100%|██████████| 500/500 [01:43<00:00,  4.84it/s]


Dataset Name:  enron , AUC Score(benchmark/combined):  72.97604967474868
Dataset Name:  enron , AUC Score (contextual):  27.23207686622321
Dataset Name:  enron , AUC Score (structural):  48.58775449546739
Dataset Name:  enron , AUC Score (joint-type):  76.78615422517862
Dataset Name:  enron , AUC Score (structure type):  49.677748239621145
Dataset Name:  enron  Best AUC Score(benchmark/combined):  78.38187463039621


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 1/500 [00:00<01:36,  5.17it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  36.759314015375516
Dataset Name:  enron , AUC Score (contextual):  9.260901699926091
Dataset Name:  enron , AUC Score (structural):  36.35607073859414
Dataset Name:  enron , AUC Score (joint-type):  42.49076127124908
Dataset Name:  enron , AUC Score (structure type):  36.58748535002715
Dataset Name:  enron  Best AUC Score(benchmark/combined):  36.759314015375516
Dataset Name:  enron , AUC Score(benchmark/combined):  54.78415138971023
Dataset Name:  enron , AUC Score (contextual):  8.435575264843557
Dataset Name:  enron , AUC Score (structural):  41.47381978501015
Dataset Name:  enron , AUC Score (joint-type):  51.62847992116284
Dataset Name:  enron , AUC Score (structure type):  41.86290483949347
Dataset Name:  enron  Best AUC Score(benchmark/combined):  54.78415138971023


  0%|          | 2/500 [00:00<01:20,  6.17it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  44.76049674748669
Dataset Name:  enron , AUC Score (contextual):  10.813008130081299
Dataset Name:  enron , AUC Score (structural):  36.93069797394362
Dataset Name:  enron , AUC Score (joint-type):  51.60877063316087


...

Dataset Name:  enron , AUC Score(benchmark/combined):  63.04479597871082
Dataset Name:  enron , AUC Score (contextual):  8.701650652870164
Dataset Name:  enron , AUC Score (structural):  39.89607172933077
Dataset Name:  enron , AUC Score (joint-type):  70.53954175905395
Dataset Name:  enron , AUC Score (structure type):  41.07690402004783
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.08722649319928


100%|█████████▉| 499/500 [01:44<00:00,  5.03it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  58.78104671791839
Dataset Name:  enron , AUC Score (contextual):  7.955161369795519
Dataset Name:  enron , AUC Score (structural):  39.85654133848516
Dataset Name:  enron , AUC Score (joint-type):  70.26730721852672
Dataset Name:  enron , AUC Score (structure type):  41.02835663036332
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.08722649319928


100%|██████████| 500/500 [01:44<00:00,  4.76it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  59.94973388527498
Dataset Name:  enron , AUC Score (contextual):  7.891106183789109
Dataset Name:  enron , AUC Score (structural):  40.40952097884777
Dataset Name:  enron , AUC Score (joint-type):  71.23675782212366
Dataset Name:  enron , AUC Score (structure type):  41.59768077828278
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.08722649319928


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 0/500 [00:00<?, ?it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  69.6732702542874
Dataset Name:  enron , AUC Score (contextual):  58.82976102488298
Dataset Name:  enron , AUC Score (structural):  39.86000891662951


  0%|          | 1/500 [00:00<03:53,  2.14it/s]

Dataset Name:  enron , AUC Score (joint-type):  34.0650406504065
Dataset Name:  enron , AUC Score (structure type):  39.63153531715405
Dataset Name:  enron  Best AUC Score(benchmark/combined):  69.6732702542874
Dataset Name:  enron , AUC Score(benchmark/combined):  73.0869308101715
Dataset Name:  enron , AUC Score (contextual):  55.45454545454545


...

Dataset Name:  enron , AUC Score (structure type):  52.26548132902648
Dataset Name:  enron  Best AUC Score(benchmark/combined):  73.0869308101715
Dataset Name:  enron , AUC Score(benchmark/combined):  60.97723240685985
Dataset Name:  enron , AUC Score (contextual):  62.611480660261144
Dataset Name:  enron , AUC Score (structural):  52.94342893941645
Dataset Name:  enron , AUC Score (joint-type):  41.89948263118995
Dataset Name:  enron , AUC Score (structure type):  52.51755614631869
Dataset Name:  enron  Best AUC Score(benchmark/combined):  73.0869308101715


100%|█████████▉| 499/500 [01:46<00:00,  4.37it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  61.471023063276164
Dataset Name:  enron , AUC Score (contextual):  62.66321754126631
Dataset Name:  enron , AUC Score (structural):  52.9302026056373
Dataset Name:  enron , AUC Score (joint-type):  41.87977334318798
Dataset Name:  enron , AUC Score (structure type):  52.50407340708344
Dataset Name:  enron  Best AUC Score(benchmark/combined):  73.0869308101715


100%|██████████| 500/500 [01:46<00:00,  4.69it/s]


Dataset Name:  enron , AUC Score(benchmark/combined):  60.799822590183325
Dataset Name:  enron , AUC Score (contextual):  62.535107169253514
Dataset Name:  enron , AUC Score (structural):  52.96413533462129
Dataset Name:  enron , AUC Score (joint-type):  41.921655580192166
Dataset Name:  enron , AUC Score (structure type):  52.53832814033482
Dataset Name:  enron  Best AUC Score(benchmark/combined):  73.0869308101715


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 1/500 [00:00<01:45,  4.74it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  38.360437610881135
Dataset Name:  enron , AUC Score (contextual):  69.71667898497167
Dataset Name:  enron , AUC Score (structural):  33.03343736067767
Dataset Name:  enron , AUC Score (joint-type):  54.76718403547671
Dataset Name:  enron , AUC Score (structure type):  33.86673527141754
Dataset Name:  enron  Best AUC Score(benchmark/combined):  38.360437610881135
Dataset Name:  enron , AUC Score(benchmark/combined):  42.53104671791839
Dataset Name:  enron , AUC Score (contextual):  73.62158167036216
Dataset Name:  enron , AUC Score (structural):  39.368405409421904
Dataset Name:  enron , AUC Score (joint-type):  47.56590293175659


  1%|          | 3/500 [00:00<01:27,  5.69it/s]

Dataset Name:  enron , AUC Score (structure type):  39.68089262403643
Dataset Name:  enron  Best AUC Score(benchmark/combined):  42.53104671791839
Dataset Name:  enron , AUC Score(benchmark/combined):  42.14518036664695
Dataset Name:  enron , AUC Score (contextual):  68.03153486080316
Dataset Name:  enron , AUC Score (structural):  38.2171694654976
Dataset Name:  enron , AUC Score (joint-type):  44.48016752894801
Dataset Name:  enron , AUC Score (structure type):  38.45434449113378
Dataset Name:  enron  Best AUC Score(benchmark/combined):  42.53104671791839
Dataset Name:  enron , AUC Score(benchmark/combined):  38.146067415730336


...

Dataset Name:  enron , AUC Score (structure type):  52.074674365644256
Dataset Name:  enron  Best AUC Score(benchmark/combined):  82.87329982259017


100%|█████████▉| 499/500 [01:47<00:00,  4.36it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  66.4229745712596
Dataset Name:  enron , AUC Score (contextual):  27.04360679970436
Dataset Name:  enron , AUC Score (structural):  47.064942784960614
Dataset Name:  enron , AUC Score (joint-type):  43.99236265089924
Dataset Name:  enron , AUC Score (structure type):  46.944849403043385
Dataset Name:  enron  Best AUC Score(benchmark/combined):  82.87329982259017
Dataset Name:  enron , AUC Score(benchmark/combined):  57.13926670609107
Dataset Name:  enron , AUC Score (contextual):  26.417836905641778
Dataset Name:  enron , AUC Score (structural):  50.36711745182543


100%|██████████| 500/500 [01:47<00:00,  4.63it/s]

Dataset Name:  enron , AUC Score (joint-type):  39.127864005912784
Dataset Name:  enron , AUC Score (structure type):  49.932586303823754
Dataset Name:  enron  Best AUC Score(benchmark/combined):  82.87329982259017


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 1/500 [00:00<01:44,  4.78it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  67.01803666469544
Dataset Name:  enron , AUC Score (contextual):  4.9051490514905165
Dataset Name:  enron , AUC Score (structural):  39.990885223163424
Dataset Name:  enron , AUC Score (joint-type):  46.07292436560729
Dataset Name:  enron , AUC Score (structure type):  40.221822027842094
Dataset Name:  enron  Best AUC Score(benchmark/combined):  67.01803666469544
Dataset Name:  enron , AUC Score(benchmark/combined):  55.703725606150215
Dataset Name:  enron , AUC Score (contextual):  13.170731707317074
Dataset Name:  enron , AUC Score (structural):  50.47198692227673
Dataset Name:  enron , AUC Score (joint-type):  27.814732692781476
Dataset Name:  enron , AUC Score (structure type):  49.59589896044746
Dataset Name:  enron  Best AUC Score(benchmark/combined):  67.01803666469544


  1%|          | 3/500 [00:00<02:17,  3.62it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  62.893258426966305
Dataset Name:  enron , AUC Score (contextual):  5.819167282581916
Dataset Name:  enron , AUC Score (structural):  48.7875365334126
Dataset Name:  enron , AUC Score (joint-type):  26.797240699679726
Dataset Name:  enron , AUC Score (structure type):  47.936521548561686
Dataset Name:  enron  Best AUC Score(benchmark/combined):  67.01803666469544
Dataset Name:  enron , AUC Score(benchmark/combined):  60.80277942046127
Dataset Name:  enron , AUC Score (contextual):  12.73712737127371
Dataset Name:  enron , AUC Score (structural):  50.478922078565425


...

100%|█████████▉| 498/500 [01:46<00:00,  4.89it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  48.92814902424601
Dataset Name:  enron , AUC Score (contextual):  87.25548164572555
Dataset Name:  enron , AUC Score (structural):  42.313171843265465
Dataset Name:  enron , AUC Score (joint-type):  63.81867455038186
Dataset Name:  enron , AUC Score (structure type):  43.14162116837702
Dataset Name:  enron  Best AUC Score(benchmark/combined):  67.01803666469544
Dataset Name:  enron , AUC Score(benchmark/combined):  48.873447664104084


100%|█████████▉| 499/500 [01:47<00:00,  4.36it/s]

Dataset Name:  enron , AUC Score (contextual):  87.41315594974132
Dataset Name:  enron , AUC Score (structural):  42.47486005845346
Dataset Name:  enron , AUC Score (joint-type):  63.798965262379895
Dataset Name:  enron , AUC Score (structure type):  43.29636299535965
Dataset Name:  enron  Best AUC Score(benchmark/combined):  67.01803666469544
Dataset Name:  enron , AUC Score(benchmark/combined):  48.58219988172679
Dataset Name:  enron , AUC Score (contextual):  87.57083025375707
Dataset Name:  enron , AUC Score (structural):  42.83989696339228
Dataset Name:  enron , AUC Score (joint-type):  63.426952451342686
Dataset Name:  enron , AUC Score (structure type):  43.633050338735956
Dataset Name:  enron  Best AUC Score(benchmark/combined):  67.01803666469544


100%|██████████| 500/500 [01:47<00:00,  4.66it/s]


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="enron")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=25)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=25)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])

print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  enron lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 1/500 [00:00<01:44,  4.78it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  28.55558840922531
Dataset Name:  enron , AUC Score (contextual):  68.92338014289233
Dataset Name:  enron , AUC Score (structural):  56.85133997126863
Dataset Name:  enron , AUC Score (joint-type):  55.52352796255236
Dataset Name:  enron , AUC Score (structure type):  56.80292332466246
Dataset Name:  enron  Best AUC Score(benchmark/combined):  28.55558840922531
Dataset Name:  enron , AUC Score(benchmark/combined):  24.228267297457123
Dataset Name:  enron , AUC Score (contextual):  73.75215570337521
Dataset Name:  enron , AUC Score (structural):  56.71471739238123
Dataset Name:  enron , AUC Score (joint-type):  47.00172456270017


  1%|          | 3/500 [00:00<01:19,  6.22it/s]

Dataset Name:  enron , AUC Score (structure type):  56.34193751250607
Dataset Name:  enron  Best AUC Score(benchmark/combined):  28.55558840922531
Dataset Name:  enron , AUC Score(benchmark/combined):  28.81431105854524
Dataset Name:  enron , AUC Score (contextual):  71.03227395910324
Dataset Name:  enron , AUC Score (structural):  56.35210779214346
Dataset Name:  enron , AUC Score (joint-type):  49.26829268292683
Dataset Name:  enron , AUC Score (structure type):  56.080858321661
Dataset Name:  enron  Best AUC Score(benchmark/combined):  28.81431105854524


...

Dataset Name:  enron , AUC Score(benchmark/combined):  54.28444707273803
Dataset Name:  enron , AUC Score (contextual):  18.461443705346145
Dataset Name:  enron , AUC Score (structural):  54.00381433595878
Dataset Name:  enron , AUC Score (joint-type):  64.18083271741808
Dataset Name:  enron , AUC Score (structure type):  54.39913672355144
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.61502069781194
Dataset Name:  enron , AUC Score(benchmark/combined):  64.95195150798344
Dataset Name:  enron , AUC Score (contextual):  31.816949987681696


100%|█████████▉| 499/500 [01:47<00:00,  5.12it/s]

Dataset Name:  enron , AUC Score (structural):  45.14048645167682
Dataset Name:  enron , AUC Score (joint-type):  58.72382360187238
Dataset Name:  enron , AUC Score (structure type):  45.663750964754314
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.61502069781194
Dataset Name:  enron , AUC Score(benchmark/combined):  67.41646954464814
Dataset Name:  enron , AUC Score (contextual):  26.850209411185023
Dataset Name:  enron , AUC Score (structural):  42.95457472630901
Dataset Name:  enron , AUC Score (joint-type):  59.09583641290958
Dataset Name:  enron , AUC Score (structure type):  43.575832070815345
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.61502069781194


100%|██████████| 500/500 [01:47<00:00,  4.65it/s]

Dataset Name:  enron , AUC Score(benchmark/combined):  50.155233589591965
Dataset Name:  enron , AUC Score (contextual):  26.125893077112593
Dataset Name:  enron , AUC Score (structural):  53.101798186951996
Dataset Name:  enron , AUC Score (joint-type):  61.01872382360186
Dataset Name:  enron , AUC Score (structure type):  53.409322623369434
Dataset Name:  enron  Best AUC Score(benchmark/combined):  75.61502069781194
